**Задача:**

Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями.

In [1]:
import random
import re

import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

1. Данные

- [Источник](https://grouplens.org/datasets/movielens/)
- Используем предобработанные для обучения данные

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1203RhVlMiIMNYRRlFNaFHGBJakgvfSsP&confirm=t&uuid=42bf735f-a423-4695-9e03-7412f5fcb692&at=ALt4Tm1C8eZaP2ZAcYBeKVVGjjyb:1690486362470')

Ознакомимся с данными

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,54,2,3.0,974918176
1,54,32,5.0,974836809
2,54,47,4.0,974837760
3,54,50,4.0,974837760
4,54,223,5.0,974840217


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040099 entries, 0 to 6040098
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 184.3 MB


In [5]:
df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Пропусков в данных нет, типы данных соответствуют содержанию. Для удобства приведем названия столбцов к snake case для удобства.

In [6]:
df.columns = [re.sub('(?=[A-Z])', '_', i).lower() for i in df]
df.sample()

,user_id,movie_id,rating,timestamp
2600149,108317,1288,4.0,942921251


Посмотрим, сколько уникальных зрителей и фильмов в датасете.

In [7]:
df[['user_id', 'movie_id']].nunique()

user_id     20000
movie_id     1000
dtype: int64

2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

In [8]:
df['watched'] = 1
df.sample()

,user_id,movie_id,rating,timestamp,watched
326963,12975,1673,2.0,920948234,1


3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.

a. Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

In [9]:
movies_values = df.movie_id.unique()
movies_values

array([    2,    32,    47,    50,   223,   260,   296,   318,   337,
         367,   541,   589,   593,   924,  1036,  1079,  1089,  1090,
        1097,  1136,  1196,  1198,  1200,  1201,  1214,  1215,  1219,
        1240,  1246,  1258,  1259,  1278,  1291,  1304,  1321,  1333,
        1358,  1370,  1374,  1387,  1584,  1967,  1994,  1997,  2021,
        2100,  2174,  2193,  2194,  2288,  2291,  2628,  2683,  2762,
        2804,  2872,  2918,  2944,  2947,  2968,  3489,    70,   110,
         480,  1210,  1270,  1356,  1544,  1580,  2455,  2791,  2858,
        2948,  2951,  3450,     1,   160,   196,   316,   329,   440,
         442,   457,   780,   788,   858,   968,  1073,  1077,  1084,
        1127,  1129,  1179,  1197,  1213,  1221,  1228,  1230,  1242,
        1247,  1272,  1288,  1307,  1345,  1372,  1373,  1375,  1376,
        1396,  1653,  1674,  1676,  1779,  1831,  1876,  1909,  1917,
        2011,  2012,  2028,  2054,  2105,  2371,  2391,  2407,  2428,
        2528,  2529,

In [10]:
users_values = df.user_id.unique()
users_values

array([    54,     91,    116, ..., 112242,  53853,  42229], dtype=int64)

b. С помощью функции random.choice [(документация)](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) сгенерируйте случайные пары пользователь * фильм

c. Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

In [11]:
# создаем пустой список для новых пар
new_pairs = []

# создаем в два раза больше пар, чем в исходном датасете
for i in range(df.shape[0] * 2):
    pair = (np.random.choice(users_values),
            np.random.choice(movies_values),
           )
    new_pairs.append(pair)

d. Среди сгенерированных пар могут быть и дубликаты, удалите их.

In [12]:
# в множестве хранятся только уникальные элементы,
# преобразуем список в множество
new_pairs = set(new_pairs)

e. Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.


In [13]:
# создаем множество пар значений из исходных данных
old_pairs = set(zip(df['user_id'].values, df['movie_id'].values))

In [14]:
# вычитаем из нового множества те значения, которые были в первом датасете
pairs = new_pairs - old_pairs

f. Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

In [15]:
len(pairs) > df.shape[0]

True

In [16]:
pairs = pd.DataFrame(random.sample(sorted(pairs), df.shape[0]))
pairs.columns = ['user_id', 'movie_id']

g. Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

In [17]:
df = pd.concat([df, pairs], ignore_index=True)

In [18]:
df.fillna(0, inplace=True)

In [19]:
df[['user_id', 'movie_id']].duplicated().sum()

0


4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

Отмасштабируем идентификаторы фильмов и пользователей таким образом, чтобы в дальнейшем находить их по индексу матрицы.

In [20]:
df.user_id = df.user_id.apply(lambda f: np.where(users_values == f)[0][0])

In [21]:
df.movie_id = df.movie_id.apply(lambda f: np.where(movies_values == f)[0][0])

In [22]:
train, test = train_test_split(df, test_size=0.01, random_state=5)

Проверим, что в тренировочный датасет попали все пользователи и фильмы

In [23]:
train[['user_id', 'movie_id']].nunique()

user_id     20000
movie_id     1000
dtype: int64

5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random [(документация)](https://numpy.org/doc/stable/reference/random/generated/numpy.random.random.html). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

In [24]:
def dummy_model(features):
    predictions = []
    for i in range(features.shape[0]):
        predictions.append(random.random())

    return predictions

In [25]:
dummy_prediction = dummy_model(test)
roc_auc_dummy = round(roc_auc_score(test.watched, dummy_prediction), 2)
print(f'Roc_auc dummy-модели: {roc_auc_dummy}')

Roc_auc dummy-модели: 0.5


6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

Сформируем матрицу user-item. По умолчанию заполним ее нулями.

In [26]:
train_matrix = np.zeros((train.user_id.nunique(), train.movie_id.nunique()))
for line in train.to_dict(orient='records'):
    train_matrix[line['user_id'], line['movie_id']] = line['watched']

Сформируем матрицы попарных косинусных расстояний

In [27]:
user_similarity = cosine_distances(train_matrix)
movie_similarity = cosine_distances(train_matrix.T)

Реализуем user-based алгоритм, перебрав несколько параметров - количество похожих пользователей, по которым делается предсказание. Предсказывать будем вероятность просмотра, а не факт - поэтому результаты предсказаний здесь и далее не будут округляться до 0 и 1. Для оценки алгоритмов будет использоваться метрика roc_auc.

In [28]:
param = [10, 20, 35, 50]

for top in param:
    top_similar_users = []
    for i in range(train.user_id.nunique()):
        neighbors = (user_similarity[i]).argsort()[1:top + 1]
        top_similar_users.append(train_matrix[neighbors])

    top_similar_users = np.array(top_similar_users)
    predicted_user_based = top_similar_users.mean(1)

    test[f'predict_user_based_{top}'] = test.apply(
        lambda f: predicted_user_based[int(f['user_id']),
                                       int(f['movie_id'])], axis = 1)
    
    roc_auc_user_based = round(roc_auc_score(test['watched'],
                                             test[f'predict_user_based_{top}']),
                                             4)
    print(f'Roc_auc user-based алгоритма ({top} похожих пользователей): {roc_auc_user_based}')

Roc_auc user-based алгоритма (10 похожих пользователей): 0.8571
Roc_auc user-based алгоритма (20 похожих пользователей): 0.8654
Roc_auc user-based алгоритма (35 похожих пользователей): 0.8676
Roc_auc user-based алгоритма (50 похожих пользователей): 0.8678


Как видно, все 4 алгоритма адекватны (метрика roc_auc выше, чем у dummy-модели со значением 0.5), лучший вариант алгоритма - при сравнении 50 похожих пользователей. Так как прирост метрики при изменении параметра с 35 до 50 пользователей невелик, остановимся на алгоритме, основанном на сравнении 50-ти ближайших пользователей со значением метрики roc_auc 0.8692.

Теперь реализуем item-based алгоритм, перебрав несколько параметров - количество похожих фильмов, по которым делается предсказание

In [34]:
param = [5, 10, 25]

for top in param:
    top_similar_movies = []
    for i in range(train.movie_id.nunique()):
        neighbors = (movie_similarity[i]).argsort()[1:top + 1]
        top_similar_movies.append(train_matrix.T[neighbors])

    top_similar_movies = np.array(top_similar_movies).T
    predicted_item_based = top_similar_movies.mean(1)

    test[f'predict_item_based_{top}'] = test.apply(
        lambda f: predicted_item_based[int(f['user_id']),
                                       int(f['movie_id'])], axis = 1)
    
    roc_auc_item_based = round(roc_auc_score(test['watched'],
                                             test[f'predict_item_based_{top}']),
                                             4)
    print(f'Roc_auc item-based алгоритма ({top} похожих фильмов): {roc_auc_item_based}')

Roc_auc item-based алгоритма (5 похожих фильмов): 0.8294
Roc_auc item-based алгоритма (10 похожих фильмов): 0.8413
Roc_auc item-based алгоритма (25 похожих фильмов): 0.8401


Все 3 item-based алгоритма адекватны (метрика roc_auc выше, чем у dummy-модели со значением 0.5), лучший вариант алгоритма - при сравнении 10 похожих фильмов, но качество все же чуть хуже, чем качество user-based алгоритма (0.8427 против 0.8692 у user-based)

Теперь реализуем алгоритм на основе матричного разложения

In [30]:
u, s, vh = svds(train_matrix, k=20)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

In [35]:
param = [10, 20, 30, 40]

for elements in param:
    u, s, vh = svds(train_matrix, k=elements)
    s_diag_matrix = np.diag(s)

    users = np.dot(u, s_diag_matrix)
    items = vh.T

    test[f'svd_predictions_{elements}'] = test.apply(
        lambda f: np.dot(users[int(f['user_id'])], items[int(f['movie_id'])]),
        axis = 1)

    roc_auc_svd = round(roc_auc_score(test['watched'],
                        test[f'svd_predictions_{elements}']),
                        4)

    print(f'Roc_auc алгоритма на основе матричного разложения: ({elements} элементов в матрицах): {roc_auc_svd}')

Roc_auc алгоритма на основе матричного разложения: (10 элементов в матрицах): 0.8735
Roc_auc алгоритма на основе матричного разложения: (20 элементов в матрицах): 0.8901
Roc_auc алгоритма на основе матричного разложения: (30 элементов в матрицах): 0.8953
Roc_auc алгоритма на основе матричного разложения: (40 элементов в матрицах): 0.8977


Лучшее значение метрики roc_auc у алгоритма на основе матричного разложения составило 0.8977 при 40 элементах в матрице.

7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

Алгоритм на основе матричного разложения проявил себя лучше всего - значение метрики roc_auc у него самое высокое из трех реализованных алгоритмов, 0.8977 при 40 элментах в матрице. При этом этот алгоритм самый быстрый как с точки зрения выполнения кода, так и с точки зрения его написания.

Лучший результат user-based агоритма - 0.8692 при предсказании на основе 50 ближайших пользователей, а для item-based - 0.8427 при предсказании на основе 10 ближайших фильмов.